In [4]:
import pandas as pd
import plotly.express as px

# Function to load data and preprocess
def load_data(url):
    df = pd.read_csv(url)
    df['Year'] = pd.to_datetime(df['Match Date']).dt.year
    return df

# Load ODI and T20 data
odi_data_url = 'https://raw.githubusercontent.com/yadus1111/NepaliCricketDataVisualization/master/data/nepal_odi_stats.csv'
t20_data_url = 'https://raw.githubusercontent.com/yadus1111/NepaliCricketDataVisualization/master/data/nepal_t20_stats.csv'

df_odi = load_data(odi_data_url)
df_t20 = load_data(t20_data_url)

# Filter data to include only matches where Nepal is the winner
nepal_wins_odi = df_odi[df_odi['Winner'] == 'Nepal']
nepal_wins_t20 = df_t20[df_t20['Winner'] == 'Nepal']

nepal_loss_odi = df_odi[df_odi['Winner'] != 'Nepal']
nepal_loss_t20 = df_t20[df_t20['Winner'] != 'Nepal']


In [28]:

# Filter to include only years where there is valid data
nepal_wins_by_year_odi = nepal_wins_odi[nepal_wins_odi.Year >= 2018].groupby('Year').size().reset_index(name='Wins')
nepal_wins_by_year_t20 = nepal_wins_t20.groupby('Year').size().reset_index(name='Wins')

# nepal_loss_by_year_odi = nepal_loss_odi[nepal_loss_odi]

# Plotting ODI wins
fig_odi = px.bar(nepal_wins_by_year_odi, x='Year', y='Wins', title='ODI Wins Over the Years')
fig_odi.show()

# Plotting T20 wins
fig_t20 = px.bar(nepal_wins_by_year_t20, x='Year', y='Wins', title='T20 Wins Over the Years')
fig_t20.show()


In [32]:
nepal_loss_odi_year = nepal_loss_odi.groupby('Year').size().reset_index(name='Loses')
nepal_odi_matches_results = pd.merge(nepal_loss_odi_year , nepal_wins_by_year_odi ,on='Year' , how='inner')
nepal_odi_matches_results

,Year,Loses,Wins
0,2018,2,1
1,2019,1,2
2,2020,2,2
3,2021,2,4
4,2022,15,6
5,2023,7,14
6,2024,3,4


In [34]:
fig = px.bar(nepal_odi_matches_results , x = 'Year' , y=['Wins' , 'Loses'] ,title="Wins and Loses of Nepal in ODI matches"  , hover_name='Year')
fig.show()

In [3]:
df_odi[df_odi['Team 1'] != 'Nepal']['Team 1'].unique() , df_t20[df_t20['Team 1'] != 'Nepal']['Team 1'].unique()

(array(['Netherlands', 'U.A.E.', 'Oman', 'Namibia', 'Scotland', 'U.S.A.',
        'India', 'Pakistan', 'Ireland', 'Zimbabwe'], dtype=object),
 array(['Hong Kong', 'Afghanistan', 'Bangladesh', 'Ireland', 'Netherlands',
        'Oman', 'Singapore', 'Kuwait', 'Malaysia', 'U.A.E.', 'Thailand',
        'Kenya', 'Canada', 'India', 'Maldives', 'Mongolia'], dtype=object))

In [34]:
# Function to calculate matchups stats for all teams
def calculate_matchup_stats_all_teams(df, team1):
    matchups_stats_all_teams = pd.DataFrame(columns=['Opponent', 'Matches Won'])

    opponents = df['Team 2'].unique()
    opponents = [team for team in opponents if team != team1]  # Exclude own team

    for opponent in opponents:
        matchups = df[((df['Team 1'] == team1) & (df['Team 2'] == opponent)) | ((df['Team 1'] == opponent) & (df['Team 2'] == team1))]
        matchups_stats = matchups.groupby('Winner').size().reset_index(name='Matches Won')
        total_matches_won = matchups_stats['Matches Won'].sum() if not matchups_stats.empty else 0
        matchups_stats_all_teams = pd.concat([matchups_stats_all_teams, pd.DataFrame({'Opponent': [opponent], 'Matches Won': [total_matches_won]})])

    return matchups_stats_all_teams

# Create doughnut chart for ODI matchups (Nepal vs Other Teams)
nepal_vs_all_teams_odi = calculate_matchup_stats_all_teams(df_odi, 'Nepal')
fig_odi = px.pie(nepal_vs_all_teams_odi, names='Opponent', values='Matches Won', title='Nepal vs Other Teams ODI Matchups')
fig_odi.show()

# Create doughnut chart for T20 matchups (Nepal vs Other Teams)
nepal_vs_all_teams_t20 = calculate_matchup_stats_all_teams(df_t20, 'Nepal')
fig_t20 = px.pie(nepal_vs_all_teams_t20, names='Opponent', values='Matches Won', title='Nepal vs Other Teams T20 Matchups')
fig_t20.show()


In [33]:
# Function to calculate matchups stats for a specific team
def calculate_matchup_stats(df, team1, team2):
    matchups = df[((df['Team 1'] == team1) & (df['Team 2'] == team2)) | ((df['Team 1'] == team2) & (df['Team 2'] == team1))]
    matchups_stats = matchups.groupby('Winner').size().reset_index(name='Matches Won')
    return matchups_stats

# Create doughnut charts for ODI matchups
for team in df_odi['Team 1'].unique():
    if team != 'Nepal':
        matchups_stats_odi = calculate_matchup_stats(df_odi, team, 'Nepal')
        fig_odi = px.pie(matchups_stats_odi, names='Winner', values='Matches Won', title=f'{team} vs Nepal ODI Matchups')
        fig_odi.show()

# Create doughnut charts for T20 matchups
for team in df_t20['Team 1'].unique():
    if team != 'Nepal':
        matchups_stats_t20 = calculate_matchup_stats(df_t20, team, 'Nepal')
        fig_t20 = px.pie(matchups_stats_t20, names='Winner', values='Matches Won', title=f'{team} vs Nepal T20 Matchups')
        fig_t20.show()

In [39]:
# Function to create a colorful bubble chart for the number of matches played in each ground
def create_colorful_bubble_chart(df, format_type):
    ground_counts = df['Ground'].value_counts().reset_index()
    ground_counts.columns = ['Ground', 'Number of Matches']

    fig = px.scatter(ground_counts, x='Ground', y='Number of Matches', size='Number of Matches', color='Number of Matches',
                     title=f'Number of {format_type} Matches Played in Each Ground',
                     labels={'Number of Matches': 'Number of Matches'},
                     size_max=40,  # Adjust the maximum bubble size as needed
                     color_continuous_scale='viridis')  # Choose a color scale

    fig.update_layout(xaxis_title='Ground', yaxis_title='Number of Matches Played')
    fig.show()

# Create colorful bubble chart for ODI matches
create_colorful_bubble_chart(df_odi, 'ODI')

# Create colorful bubble chart for T20 matches
create_colorful_bubble_chart(df_t20, 'T20')